In [1]:
import pandas as pd
from ensemble import StdForestClassifier
from category_encoders.m_estimate import MEstimateEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,classification_report
from sklearn.ensemble import RandomForestClassifier
import shap
import numpy as np
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler

In [2]:
df = pd.read_csv('data/train_transaction.csv')

In [3]:
df = df.iloc[:, :-300]
#df = df.sample(frac=0.1)
undersample = RandomUnderSampler(sampling_strategy=0.5)

In [4]:
X = df.drop(columns=['TransactionID', 'isFraud', 'TransactionDT'])
y = df[['isFraud']]

In [5]:
X,y = undersample.fit_resample(X,y)

In [6]:
X = X.fillna(-1)

In [7]:
X = MEstimateEncoder().fit_transform(X,y)

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead


In [8]:
X_tr, X_te, y_tr, y_te = train_test_split(X,y,stratify=y,test_size=0.1)

In [9]:
rf = RandomForestClassifier(n_estimators= 10)
rf = StdForestClassifier()

In [10]:
%%time
rf.fit(X_tr,y_tr)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


CPU times: user 10.3 s, sys: 134 ms, total: 10.4 s
Wall time: 10.6 s


In [11]:
rf.predict(X_tr)

0.0

In [12]:
print(classification_report(rf.predict(X_te),y_te))

TypeError: Singleton array 0.0 cannot be considered a valid collection.

In [ ]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()